* Read CSV
* Generate Table with headings
* Generate y_train data
 * Read Steering Angle
* Generate X_train data
 * Read images (center, left, right) using filename
 * Crop
 * Augment
  * Flip,resize - append(y_train, offset)
  * Noise, resize - append(y_train, offset)
  * blur, resize - append(y_train, offset)

Neural Network Implmentation
training, validation, testing split
Generator Function

Neural Network Training
Save Model

    

In [ ]:
## Learning: Important to cast to float. For example: float(line[3])
## Learning: left_angle = float(batch_sample[3])+steering_ang_correction .. for left image we add not subtract!!!
## since it is a correction


### TUNING PARAMETERS

In [ ]:
# Steering Angle Offset
# Note: Steering Angle is normalized to -1 and 1
steering_ang_correction = 2
zero_angle_keep = 0.8

### Read CSV

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
csv_headers = ["center", "left", "right", "steering", "throttle", "brake", "speed"]

data = pd.read_csv('CarSim_data/driving_log.csv', names=csv_headers)

### Create Features and Labels

In [ ]:
def i_crop(I):
    return I[55:135,:]

def i_resize(I):
    return cv2.resize(I,(64, 64),interpolation=cv2.INTER_AREA)

def i_flip(I, steering):
    return cv2.flip(I,1), -steering
    
def i_jitter(I, steering):
    I = cv2.cvtColor(I, cv2.COLOR_RGB2HSV)
    I[:,:,2] = I[:,:,2]+(np.random.uniform(-20,20))
    return cv2.cvtColor(I, cv2.COLOR_HSV2RGB), steering   

### Image Procesing Playground

In [ ]:
II = cv2.imread('./CarSim_data/IMG/left_2017_03_07_19_43_54_867.jpg')
II = cv2.cvtColor(II,cv2.COLOR_BGR2RGB)
plt.imshow(II)

### Generator

In [ ]:
import sklearn
import pandas
from sklearn.utils import shuffle
import csv

In [ ]:
samples = []
hist_angle = []
with open('./CarSim_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        if float(line[3]) == 0:
            if np.random.random() < zero_angle_keep:
                hist_angle.append(float(line[3]))
                samples.append(line)
        else:
            hist_angle.append(float(line[3]))
            samples.append(line)

In [ ]:
plt.hist(hist_angle, bins = 40);
plt.xlabel('Steering Angles')
plt.ylabel('Frequency')
plt.title('Steering Angle Histogram')

In [ ]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [ ]:
print(len(train_samples))

## Keras Model

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Convolution2D, Dense, Dropout, Flatten, Lambda, MaxPooling2D
from keras.optimizers import Adam
#from keras.layers.normalization import BatchNormalization


In [ ]:
# from keras.utils.visualize_util import plot

In [ ]:
ch, row, col = 3,64,64

In [ ]:
model = Sequential()

model.add(Lambda(lambda x: x/127.5-1., input_shape=(col, row, ch), output_shape=(col, row, ch)))


model.add(Convolution2D(24,5,5, init='glorot_uniform', subsample=(2, 2), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(36,5,5, init='glorot_uniform', subsample=(2, 2), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(48,5,5, init='glorot_uniform', subsample=(2, 2), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(64,3,3, init='glorot_uniform', border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(64,3,3, init='glorot_uniform', border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1164))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))

In [ ]:
#print(model.summary())

In [ ]:
model.compile(loss='mse', optimizer='adam')

In [ ]:
def myGenerator(samples, batch_size):
    num_samples = len(samples)
    while 1: # loop forever so generator never terminates
        shuffle(samples)
        # for logging
        batch_num_idx = 1
        for offset in range(0, num_samples, batch_size):
            
#             print('Batch Number: ', batch_num_idx, ' End')
#             print(' ')
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            
            for batch_sample in batch_samples:
                
                # Center Image ===================
                name = batch_sample[0].strip()
                center_image = i_resize(i_crop(cv2.imread(name)))
                center_image = cv2.cvtColor(center_image,cv2.COLOR_BGR2RGB)
                center_angle = float(batch_sample[3])               
                images.append(center_image)
                angles.append(center_angle)
                
                center_image_flip, center_angle_flip = i_flip(center_image, center_angle)                
                images.append(center_image_flip)
                angles.append(center_angle_flip)
                
                center_image_jitter, center_angle_jitter = i_jitter(center_image, center_angle)                
                images.append(center_image_jitter)
                angles.append(center_angle_jitter)
                
                center_image_flip_jitter, center_angle_flip_jitter = i_jitter(center_image_flip, center_angle_flip)                
                images.append(center_image_flip_jitter)
                angles.append(center_angle_flip_jitter)
                
            
            
                # Left Image =====================
                name = batch_sample[1].strip()
                left_image = i_resize(i_crop(cv2.imread(name)))
                left_image = cv2.cvtColor(left_image,cv2.COLOR_BGR2RGB)
                left_angle = float(batch_sample[3])+steering_ang_correction
                images.append(left_image)
                angles.append(left_angle)
                
                left_image_flip, left_angle_flip = i_flip(left_image, left_angle)                
                images.append(left_image_flip)
                angles.append(left_angle_flip)
                
                left_image_jitter, left_angle_jitter = i_jitter(left_image, left_angle)                
                images.append(left_image_jitter)
                angles.append(left_angle_jitter)
                
                left_image_flip_jitter, left_angle_flip_jitter = i_jitter(left_image_flip, left_angle_flip)                
                images.append(left_image_flip_jitter)
                angles.append(left_angle_flip_jitter)
                
                # Right Image
                name = batch_sample[2].strip()
                right_image = i_resize(i_crop(cv2.imread(name)))
                right_image = cv2.cvtColor(right_image,cv2.COLOR_BGR2RGB)
                right_angle = float(batch_sample[3])-steering_ang_correction
                images.append(right_image)
                angles.append(right_angle)
                
                right_image_flip, right_angle_flip = i_flip(right_image, right_angle)                
                images.append(right_image_flip)
                angles.append(right_angle_flip)
                
                right_image_jitter, right_angle_jitter = i_jitter(right_image, right_angle)                
                images.append(right_image_jitter)
                angles.append(right_angle_jitter)
                
                right_image_flip_jitter, right_angle_flip_jitter = i_jitter(right_image_flip, right_angle_flip)                
                images.append(right_image_flip_jitter)
                angles.append(right_angle_flip_jitter)
                         
            X_train = np.array(images)
#             print('X_train Shape')
#             print(X_train.shape)
#             print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
#             print(' ')
            y_train = np.array(angles)
            
            # for logging
            batch_num_idx = batch_num_idx+1
            
            yield shuffle(X_train, y_train)

In [ ]:
# compile and train the model using generator function
train_generator = myGenerator(train_samples, batch_size=256)
validation_generator = myGenerator(validation_samples, batch_size=256)

In [ ]:
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=5)

### Save Model

In [ ]:
import json

model_json = model.to_json()
with open ('model.json', 'w') as f:
    json.dump(model_json, f, indent=4, sort_keys=True, separators=(',', ':'))
    
# model.save_weights will only save the weights
model.save('model.h5')
print("Model Saved")

# **Behavioral Cloning** 

## Writeup Template

### You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

---

**Behavrioal Cloning Project**

The goals / steps of this project are the following:
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report


[//]: # (Image References)

[image1]: ./examples/placeholder.png "Model Visualization"
[image2]: ./examples/placeholder.png "Grayscaling"
[image3]: ./examples/placeholder_small.png "Recovery Image"
[image4]: ./examples/placeholder_small.png "Recovery Image"
[image5]: ./examples/placeholder_small.png "Recovery Image"
[image6]: ./examples/placeholder_small.png "Normal Image"
[image7]: ./examples/placeholder_small.png "Flipped Image"

## Rubric Points
### Here I will consider the [rubric points](https://review.udacity.com/#!/rubrics/432/view) individually and describe how I addressed each point in my implementation.  

---
### Files Submitted & Code Quality

#### 1. Submission includes all required files and can be used to run the simulator in autonomous mode

My project includes the following files:
* model.py containing the script to create and train the model
* drive.py for driving the car in autonomous mode
* model.h5 containing a trained convolution neural network 
* writeup_report.md or writeup_report.pdf summarizing the results

#### 2. Submssion includes functional code
Using the Udacity provided simulator and my drive.py file, the car can be driven autonomously around the track by executing 
```sh
python drive.py model.h5
```

#### 3. Submssion code is usable and readable

The model.py file contains the code for training and saving the convolution neural network. The file shows the pipeline I used for training and validating the model, and it contains comments to explain how the code works.

### Model Architecture and Training Strategy

#### 1. An appropriate model architecture has been employed

My model consists of a convolution neural network with both 3x3 and 5x5 filter sizes and depths between 32 and 128. This has been adopted from the [NVIDIA Paper](https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf) 

```python
model.add(Convolution2D(24,5,5, init='glorot_uniform', subsample=(2, 2), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))
```

The model includes RELU layers to introduce nonlinearity (code line 20), and 

```python
model.add(Activation('relu'))
```

The data is normalized in the model using a Keras lambda layer: 

```python
model.add(Lambda(lambda x: x/127.5-1., input_shape=(col, row, ch), output_shape=(col, row, ch)))
```



#### 2. Attempts to reduce overfitting in the model

The model contains dropout layers in order to reduce overfitting:

```python
model.add(Dropout(0.2))
```

The model was trained and validated on different data sets to ensure that the model was not overfitting (code line 10-16). The model was tested by running it through the simulator and ensuring that the vehicle could stay on the track.

#### 3. Model parameter tuning

The model used an adam optimizer, so the learning rate was not tuned manually:

```python
model.compile(loss='mse', optimizer='adam')
```

#### 4. Appropriate training data

Training data was chosen to keep the vehicle driving on the road. I used a combination of center lane driving, recovering from the left and right sides of the road.

For details about how I created the training data, see the next section.



### Model Architecture and Training Strategy

#### 1. Solution Design Approach

The overall strategy for deriving a model architecture was to ...

My first step was to use a convolution neural network model similar to the ... I thought this model might be appropriate because ...

In order to gauge how well the model was working, I split my image and steering angle data into a training and validation set. I found that my first model had a low mean squared error on the training set but a high mean squared error on the validation set. This implied that the model was overfitting. 

To combat the overfitting, I modified the model so that ...

Then I ... 

The final step was to run the simulator to see how well the car was driving around track one. The vehicle could not keep on the track after the bridge. This is because the road boundaries disappeared.
At the end of the process, the vehicle is able to drive autonomously around the track without leaving the road.

#### 2. Final Model Architecture

The final model architecture (model.py lines 18-24) consisted of a convolution neural network with the following layers and layer sizes ...


#### 3. Creation of the Training Set & Training Process

*Data Augmentation:*

Data augmentation is the process of modifying the training data in order to increase the number of samples. 

Data Flipping helped remove the bias which was generated as a result of driving in one particular direction. However for every image which was flipped, the steering wheel angle had to be inverted. E.g. if the steering angle is $+5^{\circ}$ then for the flipped image the corresponding angle is $-5^{\circ}$ 
```python
def i_flip(I, steering):
    return cv2.flip(I,1), -steering
```

Jitter helped generalized the model significantly. It may be one of the reason the vehicle was able to maneuver the turn after the bridge.

```python
def i_jitter(I, steering):
    I = cv2.cvtColor(I, cv2.COLOR_RGB2HSV)
    I[:,:,2] = I[:,:,2]+(np.random.uniform(-20,20))
    return cv2.cvtColor(I, cv2.COLOR_HSV2RGB), steering 
```

#### Reducing Zero Band:
It was observed that many of the images had zero steering angle associated with them. Hence even if the vehicle was headed towards the outside of the road, it did not learn to recover because it would predict the angle as $^{\circ}$

```python
if float(line[3]) == 0:
            if np.random.random() < zero_angle_keep:
                hist_angle.append(float(line[3]))
                samples.append(line)
        else:
            hist_angle.append(float(line[3]))
            samples.append(line)
```

<p align="center">
  <img src="writeup_data/hist_zero.jpg" alt="Histogram" height="80"/>
</p>

The model was compiled using the Mean Square Error as the loss and the Adam optimizer.
```python
model.compile(loss='mse', optimizer='adam')
```

### Generators
The concept of generators was introduced as a result of this project. Generators helped control the number of samples sent through a batch. This enabled learning on GPUs with limited memory.

5 number of Epochs were sufficient for training.

### Train-Validation Split

The samples were split into training (80%) and validation (20%) sets randomly. The validation set was used to prevent overfitting.

```python
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
```

To capture good driving behavior, I first recorded two laps on track one using center lane driving. Here is an example image of center lane driving:

![alt text][image2]

I then recorded the vehicle recovering from the left side and right sides of the road back to center so that the vehicle would learn to .... These images show what a recovery looks like starting from ... :

<p align="left">
  <img src="writeup_data/left.jpg" alt="Left" height="80"/>
</p>
<p align="center">
  <img src="writeup_data/center.jpg" alt="Center" height="80"/>
</p>
<p align="right">
  <img src="writeup_data/right.jpg" alt="Right" height="80"/>
</p>

Then I repeated this process on track two in order to get more data points.

To augment the data sat, I also flipped images and angles thinking that this would ... For example, here is an image that has then been flipped:

![alt text][image6]
![alt text][image7]

Etc ....

After the collection process, I had X number of data points. I then preprocessed this data by ...


I finally randomly shuffled the data set and put Y% of the data into a validation set. 

I used this training data for training the model. The validation set helped determine if the model was over or under fitting. The ideal number of epochs was Z as evidenced by ... I used an adam optimizer so that manually training the learning rate wasn't necessary.

For a very long time the car kept going out no matter how much data I used. Later I realized that I was subtracting the steering angle correction instead of adding. After making the correction it worked.
```python
left_angle = float(batch_sample[3]) + steering_ang_correction
```

In [ ]:
I = cv2.imread('./CarSim_data/IMG/right_2017_02_18_17_18_32_266.jpg')
I = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)
Iflip, s = i_flip(I,5)

plt.subplot(1,2,1)
plt.imshow(I)

plt.subplot(1,2,2)
plt.imshow(Iflip)

plt.imsave(ori_flip)